# Regression models for wine quality prediction

We are working with **UCI white wine quality dataset**; related to Portuguese "Vinho Verde" wine, [more info here](http://www.vinhoverde.pt/en/). Things like Whine brand, grape type or price are not available in the dataset due to privacy issues.

First of all, like any ML project **we have started based on EDA** (Exploratory Data Analysis) to find some insights, ideas and to understand what we have, where we want to go and how. For better understanding EDA has been separated and **you can find it [here](here).**

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

In [3]:
df = pd.read_csv('data/winequality-white.csv', sep=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


With pd.info() we can easily see the columns we have a the data type, **we are trying to predict the quality** so this column it's going to be our target:

In [5]:
y = df['quality']
X = df.drop('quality', axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=94)

As usual (depending on the project), as we are going to try some regressors we don't want biggest scaled features to weight more in our model, so **let's scale all of them**. 

If a feature's variance is orders of magnitude more than the variance of others, **that particular feature will dominate other in the dataset** and we don't want this in our model.

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Models
### Linear Regression

A good approach **always is to start simple**; normally all ML masters agree that simple models like Linear Regressors, Logistic Regressors (for classification) or k-means performs really well but need more feature and data work.

So let start simple and then we can become more complex to explore different solutions and approaches:

In [7]:
lin = LinearRegression()
lin.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
lin_pred_train = lin.predict(X_train_scaled)
lin_pred_test = lin.predict(X_test_scaled)

print("MSE for train set: %.4f" % mean_squared_error(y_train, lin_pred_train))
print("MSE for test set: %.4f" % mean_squared_error(y_test, lin_pred_test))

MSE for train set: 0.5501
MSE for test set: 0.6023


Our LinealRegression doesn't perform so bad; 0.6 points in a 0-10 scale range for wine quality having only 5k samples is a **good start point.** 

A really interesting thing about regression is to explore the **features coefficients** so we can see and understand which of them are more important for our output value.

In [9]:
lin_coeff = pd.DataFrame(
    {'coef': lin.coef_, 'coef_abs': np.abs(lin.coef_)}, index=X.columns)
lin_coeff.sort_values(by='coef_abs', ascending=False)

,coef,coef_abs
density,-0.664905,0.664905
residual sugar,0.525293,0.525293
volatile acidity,-0.191563,0.191563
alcohol,0.131448,0.131448
pH,0.129702,0.129702
fixed acidity,0.105801,0.105801
sulphates,0.080057,0.080057
free sulfur dioxide,0.058389,0.058389
chlorides,0.008659,0.008659
total sulfur dioxide,0.006759,0.006759


### Lasso Regression
After trying with LinearRegression **we are going to explore Lasso Regression**, Lasso comes from **L**east **a**bsolute **s**hrinkage and **s**election **o**perator and is a method where we select features and use regularization to improve model's performance. 

In [10]:
lasso = Lasso(alpha=0.01, random_state=94)
lasso.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=94,
      selection='cyclic', tol=0.0001, warm_start=False)

Let's see if we improve our LinearRegression

In [11]:
lasso_pred_train = lasso.predict(X_train_scaled)
lasso_pred_test = lasso.predict(X_test_scaled)

print("MSE for train set: %.4f" %
      mean_squared_error(y_train, lasso_pred_train))
print("MSE for test set: %.4f" % mean_squared_error(y_test, lasso_pred_test))

MSE for train set: 0.5565
MSE for test set: 0.5916


In [12]:
lasso_coeff = pd.DataFrame(
    {'coef': lasso.coef_, 'coef_abs': np.abs(lasso.coef_)}, index=X.columns)
lasso_coeff.sort_values(by='coef_abs', ascending=False)

,coef,coef_abs
alcohol,0.340011,0.340011
residual sugar,0.224591,0.224591
density,-0.206103,0.206103
volatile acidity,-0.189423,0.189423
free sulfur dioxide,0.054941,0.054941
sulphates,0.043467,0.043467
pH,0.039923,0.039923
chlorides,-0.003110,0.003110
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000


We have begun with 0.01 as alpha value; take in mind that **if we use alpha=0 we will have a LinearRegression** because we will multiply the regularization parameter by zero. 

We are going to use **cross-validation lasso estimator to find the best alpha** value to fit our data and see if it improves the mean squared error we got previously

In [13]:
alphas = np.logspace(-6, 3, 300)
lasso_cv = LassoCV(random_state=94, cv=5, alphas=alphas)
lasso_cv.fit(X_train_scaled, y_train)
lasso_cv.alpha_

0.0001469684386112446

In [14]:
lasso_pred_train = lasso_cv.predict(X_train_scaled)
lasso_pred_test = lasso_cv.predict(X_test_scaled)

print("MSE for train set: %.4f" %
      mean_squared_error(y_train, lasso_pred_train))
print("MSE for test set: %.4f" % mean_squared_error(y_test, lasso_pred_test))

MSE for train set: 0.5501
MSE for test set: 0.6018


In [16]:
lasso_cv_coef = pd.DataFrame({'coef': lasso_cv.coef_, 'coef_abs': np.abs(lasso_cv.coef_)},
                             index=df.columns.drop('quality'))
lasso_cv_coef.sort_values(by='coef_abs', ascending=False)

,coef,coef_abs
density,-0.655494,0.655494
residual sugar,0.519243,0.519243
volatile acidity,-0.191423,0.191423
alcohol,0.135504,0.135504
pH,0.127860,0.127860
fixed acidity,0.103432,0.103432
sulphates,0.079426,0.079426
free sulfur dioxide,0.058675,0.058675
chlorides,0.008036,0.008036
total sulfur dioxide,0.006016,0.006016


### Random Forest
Finally we will go with a more complex model, the problem with Random Forest is that **we loose the interpretability** we had with simpler models we used before. Also it is a bit computationally expensive.

In the other hand, we normally win in **performance and results**. One good thing about Random Forest is that **it holds outliers really well due to random sampling**, in our case we don't have a balanced dataset so outliers could affect model's performance.

In [17]:
rfr = RandomForestRegressor(random_state=94, n_estimators=10)
rfr.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=94, verbose=0,
                      warm_start=False)

In [18]:
rfr_pred_train = rfr.predict(X_train_scaled)
rfr_pred_test = rfr.predict(X_test_scaled)

print("MSE for train set: %.4f" % mean_squared_error(y_train, rfr_pred_train))
print("MSE for test set: %.4f" % mean_squared_error(y_test, rfr_pred_test))

MSE for train set: 0.0792
MSE for test set: 0.4522


It's a really good result, being our fist Random Forest attempt; maybe it's a good idea to use **GridSearch to tune model's hyperparameters** and see if we can go a bit further.

In [19]:
rfr_param_grid = {'max_depth': list(range(6, 22)),
                  'max_features': list(range(4, 12)),
                  'n_estimators': list(range(5, 10))}

rfr_grid = GridSearchCV(RandomForestRegressor(random_state=94),
                        rfr_param_grid,
                        scoring='neg_mean_squared_error',
                        n_jobs=-1,
                        cv=5,
                        verbose=True)

rfr_grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 640 candidates, totalling 3200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 345 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 845 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done 1545 tasks      | elapsed:   56.8s
[Parallel(n_jobs=-1)]: Done 2445 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 3193 out of 3200 | elapsed:  2.3min remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 3200 out of 3200 | elapsed:  2.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=94,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [6, 7, 8, 9, 10, 11, 1

In [20]:
rfr_grid.best_score_, rfr_grid.best_params_

(-0.4374759144261164, {'max_depth': 20, 'max_features': 4, 'n_estimators': 9})

In [21]:
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(
    rfr_grid.best_estimator_,
    X_train_scaled, y_train,
    scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test,
                                                             rfr_grid.predict(X_test_scaled)))

c:\users\juanlu\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Mean squared error (cv): 0.467
Mean squared error (test): 0.444


This way we got the best score of our tested models, getting better score in Random Forest can tell us that **wine quality is non-linear**, so other models like LightGM or XGBoost can perform well. 

In the next notebook we will explore these kind of solutions and **also stacking models to improve performance** in our project.